Data_Engineering_TIL_(20190928)

#### [study program]


'19.9.28.(토) AWS 한국사용자모임 핸즈온행사


URL : https://awskrug.github.io


#### [실습목표]

AWS Glue 서비스를 중심으로 한 데이터 ETL 처리 아키텍처 구현 및 실제 데이터 처리

<img src="00.png" width="1000" />

#### [실습 간 사용할 데이터]

<img src="0.png" width="1000" />

#### [실습요약]

실습 1. AWS EMR을 통한 ETL 작업수행

실습 2. AWS Glue를 통한 ETL 작업수행

#### [실습상세]

#### 실습 1. AWS EMR을 통한 ETL 작업수행

#### STEP 1) AWS EMR 클러스터 구현

step 1-1) 아래 그림과 같이 설정 후 클러스터 생성

** 참고사항

JAR location
s3://ap-northeast-2.elasticmapreduce/libs/script-runner/script-runner.jar

Arguments
s3://skawscamp/mysql.sh

<img src="1.png" width="1000" />

step 1-2) EMR 보안취약점 개선을 위한 Block public access 설정

<img src="1-1.png" width="1000" />

step 1-3) zeppelin 사용을 위한 보안그룹 설정

보안그룹 콘솔로 접속해서 ElasticMapReduce-master 선택 -> inbound role edit 클릭 -> 아래 그림과 같이 All TCP 추가 및 source는 My IP로 설정 -> save

<img src="9.png" width="1000" />

#### STEP 2) AWS 일부 자원 컨트롤을 위한 EC2 생성 및 권한부여

아래 그림과 같이 EC2 생성 후 IAM role 부여

<img src="10.png" width="1000" />

<img src="11.png" width="1000" />

#### STEP 3) 실습데이터 적재 및 활용을 위한 RDS 생성

아래 그림과 같이 설정 후  RDS 생성

<img src="18.png" width="1000" />

#### STEP 4) 생성한 EC2로 SSH 접속 후 필요한 자료를 S3에 적재

아래 그림과 같이 SSH로 생성한 EC2에 접속하여 실습에 필요한 자료들을 S3에 적재한다.

<img src="25.png" width="1000" />

** 노트북을 s3로 카피하는 작업은 EC2에서 datalab 폴더로 이동한 다음에 해준다.

#### STEP 5) AWS Glue 콘솔로 이동하여 아래 그림과 같이 데이터 카탈로그 메뉴에서 create database 수행

<img src="32.png" width="1000" />

#### STEP 6) EMR을 이용한 데이터 ETL 수행

아래 그림과 같이 제플린에 접속하여 ETL 수행에 필요한 노트북을 import

<img src="33.png" width="1000" />

step 6-1) 아래 그림과 같이 S3에 적재된 실습데이터를 spark를 이용하여 처리하고, 위에서 생성했던 RDS에 적재하는 작업 수행

** url 변경할때 aws -> datalab으로 바꿔줘야 함

<img src="41.png" width="1000" />

step 6-2)  아래 그림과 같이 S3에 적재된 실습데이터(로그데이터)를 spark를 이용하여 처리해본다.

<img src="34.png" width="1000" />

#### 실습 2. AWS Glue를 통한 ETL 작업수행

#### STEP 1) Glue Dev endpoint 생성

AWS Glue에서는 일반적으로 ETL 작업을 할 때 데이터스토어로 Amazon S3를 많이 쓰는데, 과도한 트래픽 비용을 막기 위해 VPC 엔드포인트를 이용하는 것이 좋다. 만약 Dev endpoint를 생성하고자 하는 VPC에 VPC endpoint가 존재하고, 라우팅 처리되어 있다면, 이 과정은 SKIP해도 무방하다.

<img src="42.png" width="1000" />

step 1-1) S3 endpoint 생성

AWS 콘솔에서 Amazon VPC  -> endpoint -> create endpoint

아래 그림과 같이 설정 후 생성

<img src="46.png" width="1000" />

step 1-2) 보안그룹에서 default 그룹을 아래 그림과 같이 변경설정

<img src="47.png" width="1000" />

위와 같이 완료되면 아래 그림과 같이 default 보안그룹에 대해서 설정을 추가

소스는 ec2의 프라이빗 아이피 기입

<img src="50.png" width="1000" />

step 1-3) Glue 서비스로 이동하여 dev endpoint -> add endpoint 클릭

아래 그림과 같이 설정 후 생성

<img src="48.png" width="1000" />

#### STEP 2) RDS에 소스 데이터를 import

위에서 생성한 EC2 쉘로 돌아가서 아래 그림과 같이 명령어를 실행한다.

<img src="51.png" width="1000" />

#### STEP 3) 글루 크롤러가 RDS에 접근하여 컬럼을 가져올 수  있도록 DB Connection 생성

<img src="52.png" width="1000" />

#### STEP 4) Crawler connections를 통해 RDS에 접근하여 컬럼을 게더링

이미 만들어진 AWS Glue Data Catalog의 테이블 목록을 소스로 지정할수도 있다. 이렇게 하게 되면 이전 크롤러가 지정한 데이터 경로를 소스로 가져와서 데이터를 스캔한 뒤 새로운 Catalog에 새 테이블을 만들 수 있다.

실습에서는 새로운 Catalog를 만든다.

<img src="57.png" width="1000" />

#### STEP 5) S3를 타겟으로 크롤링 수행 

앞서 Database를 크롤링한 것과 방법은 동일함.

<img src="68.png" width="1000" />

이후 Athena 혹은 Redshift spectrum에서 조회하려면 아래와 같이 파티션에 조건을 걸어 제한을 둘 수 있다.

Athena로 예를들면, `SELECT * FROM user_event_logs_2019 where partition_0='09' and partition_1='22'`

<img src="69.png" width="1000" />

위에 그림과 같이 전체 테이블을 질의하는 쿼리와 파티셔닝된 테이블을 조건절로 준 쿼리와 수행시간, 데이터 스캔 사이즈가 확연히 차이가 나는 것을 알 수 있다. 

#### STEP 6) Glue Jupyter notebook 활용한 ETL 작업 수행

앞서 만들어둔 AWS Glue Dev endpoint를 통해 Jupyter notebook을 만들어 본다.

주피터 노트북에서 개발자들은 간편하게 Crawler가 만든 테이블을 조회하거나, ETL 테스트를 할 수 있다. 

step 6-1) 글루 노트북 셋팅

아래그림과 같이 노트북을 생성한다.

<img src="70.png" width="1000" />

생성이 완료되면 아래 그림과 같은 방법으로 노트북에 접속한다.

<img src="71.png" width="1000" />

노트북에 접속하면 아래그림과 같이 오픈 노트북랩을 클릭해서 접속한다.

<img src="72.png" width="1000" />

아래 그림과 같이 깃클론을 하여 필요한 노트북을 가져온다.

<img src="73.png" width="1000" />

step 6-2) 아래 그림과 같이 노트북을 이용하여 ETL 작업을 수행한다.

<img src="74.png" width="1000" />

#### STEP 7) Glue job을 이용한 ETL 작업수행

step 7-1) 크롤러 생성 및 크롤링 수행

Database에 들어있는 Order 테이블을 Amazon S3에 있는 Order_info 테이블에 필요한 부분만 넘겨 보려고한다.

먼저 타겟 테이블인 Order_info를 크롤러로 정의한다.

<img src="75.png" width="1000" />

step 7-2) 글루 잡을 이용한 ETL 작업수행

<img src="76.png" width="1000" />

<img src="84.png" width="1000" />